In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [25]:
s = ['A191RL']

df = nipa_df(retrieve_table('T10502')['Data'], s)

df.to_csv(data_dir / 'gdp.csv', index_label='date')

txt = f'{df.index[-1].year} Q{df.index[-1].quarter}: {df["A191RL"].iloc[-1]}\%'

write_txt(data_dir / 'gdp.txt', txt)

In [4]:
s = ['DPCERY', 'A006RY', 'A822RY', 'A019RY']

(nipa_df(retrieve_table('T10502')['Data'], s)
 .to_csv(data_dir / 'comp.csv', index_label='date'))

In [5]:
s = ['A939RC', 'A939RX']

df = nipa_df(retrieve_table('T70100')['Data'], s)
df['value'] = (df['A939RX'] / df['A939RX'].iloc[-1])  * df['A939RC'].iloc[-1]
df[['value']].to_csv(data_dir / 'gdppc.csv', index_label='date')

In [4]:
s = ['A261RX', 'W256RX']
rgdi = nipa_df(retrieve_table('T11706')['Data'], s).dropna()

s = ['A261RC', 'A4002C', 'W056RC', 'A107RC', 'W271RC', 'A262RC']
df = nipa_df(retrieve_table('T11000')['Data'], s).dropna()

# Calculate indirect taxes net of transfers
df['indirect'] = df['W056RC'] - df['A107RC']
df = df.drop(['A107RC', 'W056RC'], axis=1)

# Calculate GDI deflator from real GDI series
deflator = rgdi['A261RX'] / df['A261RC']
deflator = deflator / deflator.iloc[-1]
df = df.multiply(deflator, axis=0)

# Calculate contributions to growth
dft = df.diff()
dft = dft.div(dft['A261RC'], axis=0)
contr = dft.multiply((((df['A261RC'].pct_change() + 1) ** 4) - 1) * 100, axis=0)
contr.round(2).to_csv(data_dir / 'gdi.csv', index_label='date')